### requirements

In [ ]:
import pyodbc
from IPython.display import clear_output
# [x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]
# if you see [], you may need to install the 64-bit ACE drivers
# https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access
# 64bit drivers : https://www.microsoft.com/en-us/download/confirmation.aspx?id=13255

from osgeo import ogr, osr
sr = osr.SpatialReference()
sr.ImportFromEPSG(4326)
sr.MorphToESRI()

### execute the stored access query
`export_shps` is the name of the stored query in the Access Db

In [ ]:
connStr = (
    r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=Z:\dev\nepal-import\NepalInfo2016_for_python.accdb;"
    )
cnxn = pyodbc.connect(connStr)
sql = """\
SELECT layer_shp, layer_shx, layer_dbf, layer_nid, layer_name, area_id FROM export_shps 
"""
crsr = cnxn.execute(sql)

# export just a subset for testing
rows = crsr.fetchmany(20)

# uncomment this to get crazy and export them all!!
# rows = crsr.fetchall()



### loop through the results
the `out_file` will have an ID at the end, after the double-underscore ( `__` ). use this to join up to the raw data from the other stored access query

In [ ]:
row_count = len(rows)

output_folder = 'shapes'

for i, row in enumerate(rows):

    shp = row[0]
    shx = row[1]
    dbf = row[2]
    
    shp_name = row[4]
    
    area_id = row[5]
    
    out_file = '{}/{}__{}'.format(output_folder, shp_name, area_id)
    
    print ('adding {} of {}'.format(i, row_count))
    clear_output(wait=True)
    
    with open('{}.shp'.format(out_file), 'wb') as writer:
        writer.write(shp)
    with open('{}.shx'.format(out_file), 'wb') as writer:
        writer.write(shx)
    with open('{}.dbf'.format(out_file), 'wb') as writer:
        writer.write(dbf)
    with open('{}.prj'.format(out_file), 'w') as writer:
        writer.write(sr.ExportToWkt())
    
print ('done')